In [ ]:
import json
import logging
import re
import unicodedata

from minio import Minio
from pyspark.sql import DataFrame, SparkSession

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [3]:
minio_connection = ""  

In [4]:
# carregar para funcionar
try:
    minio_conn = json.loads(minio_connection)
except json.JSONDecodeError:
    with open('../variables/minio_connection.json', "r") as minio_connection_file:
        minio_conn = json.loads(minio_connection_file.read())

In [5]:
class LazySparkSession:
    packages = [
        "io.delta:delta-spark_2.13:4.0.0",
        "org.apache.hadoop:hadoop-aws:3.4.0",
        "com.amazonaws:aws-java-sdk-bundle:1.12.787",
    ]

    def __init__(self, access_key, secret_key, endpoint):
        self._access_key = access_key
        self._secret_key = secret_key
        self._endpoint = endpoint
        

    def start(
        self,
        app_name: str = "Airflow Spark Delta Minio App",
        executor_memory: str = "1g",
        driver_memory: str = "1g",
        driver_maxresultsize: str = "1g",
        master_url: str = "local[*]",
    ):

        builder = (
            SparkSession
            .Builder()
            .appName(app_name)
            .config("spark.hadoop.fs.s3a.access.key", self._access_key)
            .config("spark.hadoop.fs.s3a.secret.key", self._secret_key)
            .config("spark.hadoop.fs.s3a.endpoint", self._endpoint)
            .config("spark.hadoop.delta.enableFastS3AListFrom", "true")
            #
            .config("spark.executor.memory", executor_memory)
            .config("spark.driver.memory", driver_memory)
            .config("spark.driver.maxResultSize", driver_maxresultsize)
            #
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            #
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
            #
            .config("spark.jars.packages", ",".join(self.packages))
            .master(master_url)
        )

        return builder.getOrCreate()

In [6]:
spark = LazySparkSession(
    access_key=minio_conn.get("access_key"), 
    secret_key=minio_conn.get("key"), 
    endpoint=minio_conn.get("endpoint")
).start()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/08 14:05:03 WARN Utils: Your hostname, DESKTOP-EDEM2DH, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/08/08 14:05:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/edcarlos/projeto-lakehouse/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/edcarlos/.ivy2.5.2/cache
The jars for the packages stored in: /home/edcarlos/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-458329e1-f14b-4713-874b-e32e242b48ec;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central

In [7]:
s3_client = None

try:
    endpoint_raw = minio_conn["endpoint"]
    access_key = minio_conn["access_key"]
    secret_key = minio_conn["key"]

    endpoint_sem_http = endpoint_raw.replace("http://", "").replace("https://", "")
    is_secure = endpoint_raw.startswith("https")

    s3_client = Minio(
        endpoint=endpoint_sem_http,
        access_key=access_key,
        secret_key=secret_key,
        secure=is_secure
    )

    logging.info("Cliente MinIO criado com sucesso.")

except KeyError as e:
    logging.error(f"Erro de configuração: chave ausente - {e}")
except Exception as e:
    logging.error(f"Erro ao inicializar o cliente MinIO: {e}")

2025-08-08 14:05:13,338 - INFO - Cliente MinIO criado com sucesso.


In [8]:
def remover_acentos(txt: str) -> str:
    """
    Remove acentos mantendo a letra original.
    """
    # Normaliza para forma NFD (separa acentos das letras)
    nfkd = unicodedata.normalize('NFD', txt)
    # Remove os caracteres de acento (categoria Mn = Mark, Nonspacing)
    return "".join([c for c in nfkd if unicodedata.category(c) != 'Mn'])

In [9]:
def padronizar_nome_coluna(nome: str) -> str:
    """
    Padroniza o nome da coluna:
    - Remove acentos sem apagar letras
    - Converte para minúsculas
    - Remove caracteres especiais
    - Substitui espaços por "_"
    """
    nome = nome.strip().lower()
    nome = remover_acentos(nome)
    nome = re.sub(r"[^\w\s]", "", nome)  # Mantém letras, números e espaços
    nome = re.sub(r"\s+", "_", nome)     # Substitui espaços por _
    return nome

In [10]:
def padronizar_colunas(df: DataFrame) -> DataFrame:
    """
    Aplica padronização em todas as colunas do DataFrame.
    """
    novas_colunas = [padronizar_nome_coluna(c) for c in df.columns]
    return df.toDF(*novas_colunas)

In [11]:
# Configurar os parâmetros para o arquivo que deseja processar
bucket = "landing"
pasta = "cno/cno"
nome_arquivo = "cno_cnaes.csv"

input_path = f"s3a://{bucket}/{pasta}/{nome_arquivo}"
output_path = f"s3a://bronze/cno/cno/cno_cnaes_delta/"

logger.info(f"Lendo arquivo: {input_path}")

2025-08-08 14:05:13,374 - INFO - Lendo arquivo: s3a://landing/cno/cno/cno_cnaes.csv


In [ ]:
# Ler CSV com header
df = (
    spark.read
    .option("header", True)
    .option("encoding", "ISO-8859-1") 
    .csv(input_path)
)
logger.info(f"Colunas originais: {df.columns}")

25/08/08 14:05:14 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
2025-08-08 14:05:17,229 - INFO - Colunas originais: ['CNO', 'CNAE', 'Data de registro']


In [13]:
# Padronizar nomes das colunas
df = padronizar_colunas(df)

logger.info(f"Colunas padronizadas: {df.columns}")

2025-08-08 14:05:17,250 - INFO - Colunas padronizadas: ['cno', 'cnae', 'data_de_registro']


In [14]:
# Salvar em Delta na camada bronze
df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(output_path)

logger.info(f"Arquivo {nome_arquivo} processado e salvo em Delta no caminho: {output_path}")

# Para verificar, conta linhas
count = df.count()
logger.info(f"Total de linhas: {count}")

25/08/08 14:05:18 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/08/08 14:05:22 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/08/08 14:05:24 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to cno/cno/cno_cnaes_delta/part-00007-27544923-c618-4a95-8e9c-18318e003a21-c000.snappy.parquet. This is Unsupported
2025-08-08 14:05:26,470 - INFO - Arquivo cno_cnaes.csv processado e salvo em Delta no caminho: s3a://bronze/cno/cno/cno_cnaes_delta/
2025-08-08 14:05:27,116 - INFO - Total de linhas: 3287104                       


In [15]:
df.show(20, truncate=False)

+------------+-------+----------------+
|cno         |cnae   |data_de_registro|
+------------+-------+----------------+
|010010092278|4120400|2022-05-17      |
|010010144970|4120400|2023-05-22      |
|010010199274|4299501|2022-12-13      |
|010010251669|4120400|2020-09-04      |
|010010287967|4120400|2019-06-24      |
|010010627767|4299599|2022-01-14      |
|010010840867|4399103|2020-07-08      |
|010010949160|4120400|2023-02-23      |
|010010962365|4120400|2020-11-10      |
|010010962365|4399103|2020-11-10      |
|010010965467|4120400|2024-04-05      |
|010011030062|4399103|2019-09-03      |
|010011090267|4399103|2024-11-06      |
|010011090462|4399103|2021-05-11      |
|010011110269|4120400|2020-08-04      |
|010011141767|4399103|2024-03-26      |
|010011146566|4399103|2022-10-04      |
|010011151965|4399103|2021-03-18      |
|010011180662|4120400|2024-01-23      |
|010011180764|4120400|2020-06-18      |
+------------+-------+----------------+
only showing top 20 rows


In [16]:
spark.stop()